# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12558782133174021529
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14357954560
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2926767800683889858
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [5]:
# Add-on : Check framework versions being used
import tensorflow as tf
import keras as K
import sys
print("Python", sys.version)
print("Tensorflow version", tf.__version__)
print("Keras version", K.__version__)

Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Tensorflow version 2.14.0
Keras version 2.14.0


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [8]:
# Remark : Encoding UTF-8 has been fixed in helper.py

# Load English data
english_sentences = helper.load_data('small_vocab_en')
# Load French data
french_sentences = helper.load_data('small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [64]:
for sample_i in range(2,10):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 3:  california is usually quiet during march , and it is usually hot in june .
small_vocab_fr Line 3:  california est généralement calme en mars , et il est généralement chaud en juin .
small_vocab_en Line 4:  the united states is sometimes mild during june , and it is cold in september .
small_vocab_fr Line 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .
small_vocab_en Line 5:  your least liked fruit is the grape , but my least liked is the apple .
small_vocab_fr Line 5:  votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .
small_vocab_en Line 6:  his favorite fruit is the orange , but my favorite is the grape .
small_vocab_fr Line 6:  son fruit préféré est l'orange , mais mon préféré est le raisin .
small_vocab_en Line 7:  paris is relaxing during december , but it is usually chilly in july .
small_vocab_fr Line 7:  paris est relaxant en décembre , mais il est généralement froid en juillet .
small_vocab_en Line 

From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [15]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [17]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement

    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

#tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [18]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        # Find the length of the longest sequence/sentence
        length = max([len(seq) for seq in x])

    return pad_sequences(sequences=x, maxlen=length, padding='post')



# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [19]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [24]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [25]:
# Add-on : Collect logs for Tensorboard
from keras.callbacks import TensorBoard
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=1, write_graph=True)

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [26]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers

    input_seq = Input(shape=input_shape[1:])
    rnn = GRU(units=english_vocab_size, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
    return model

#tests.test_simple_model(simple_model)

# Pad and Reshape the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 9s 26ms/step - loss: 2.7039 - accuracy: 0.4629 - val_loss: 2.0713 - val_accuracy: 0.5280
Epoch 2/10
108/108 [==============================] - 2s 20ms/step - loss: 1.7915 - accuracy: 0.5658 - val_loss: 1.6104 - val_accuracy: 0.5827
Epoch 3/10
108/108 [==============================] - 2s 22ms/step - loss: 1.5243 - accuracy: 0.5967 - val_loss: 1.4479 - val_accuracy: 0.6088
Epoch 4/10
108/108 [==============================] - 2s 22ms/step - loss: 1.3906 - accuracy: 0.6196 - val_loss: 1.3367 - val_accuracy: 0.6303
Epoch 5/10
108/108 [==============================] - 2s 21ms/step - loss: 1.2930 - accuracy: 0.6371 - val_loss: 1.2494 - val_accuracy: 0.6448
Epoch 6/10
108/108 [==============================] - 2s 21ms/step - loss: 1.2152 - accuracy: 0.6502 - val_loss: 1.1815 - val_accuracy: 0.6534
Epoch 7/10
108/108 [==============================] - 2s 21ms/step - loss: 1.1574 - accuracy: 0.6599 - val_loss: 1.1341 - val_accuracy: 0.6620

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [28]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    #print("Debug input_shape =" , input_shape, "  Input length=", input_shape[1:][0])
    #print("Debug output_sequence_length =" , output_sequence_length)
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)

    # Hyperparameters
    embedding_size = 128
    rnn_cells = 200
    dropout = 0.0
    learning_rate = 1e-3

    # Sequential Model
    #from keras.models import Sequential
    #model = Sequential()
    #model.add(Embedding(english_vocab_size, embedding_size, input_length=input_shape[1:][0]))
    #model.add(GRU(rnn_cells, dropout=dropout, return_sequences=True))
    #model.add(Dense(french_vocab_size, activation='softmax'))
    #print(model.summary())

    # model's Functional equivalent
    input_seq = Input(shape=input_shape[1:])

    embedded_seq = Embedding(input_dim = english_vocab_size,
                             output_dim = embedding_size,
                             input_length=input_shape[1:][0])(input_seq)

    rnn = GRU(units=rnn_cells, dropout=dropout, return_sequences=True)(embedded_seq)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    #print(model.summary())

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model

#tests.test_embed_model(embed_model)


# Pad the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#print("Debug tmp_x shape=", tmp_x.shape )


# Train the neural network
embed_rnn_model = embed_model(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)


embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 8s 50ms/step - loss: 3.1994 - accuracy: 0.4232 - val_loss: 2.4785 - val_accuracy: 0.4906
Epoch 2/10
108/108 [==============================] - 3s 26ms/step - loss: 1.9735 - accuracy: 0.5635 - val_loss: 1.4696 - val_accuracy: 0.6425
Epoch 3/10
108/108 [==============================] - 3s 23ms/step - loss: 1.2186 - accuracy: 0.7010 - val_loss: 0.9997 - val_accuracy: 0.7550
Epoch 4/10
108/108 [==============================] - 3s 26ms/step - loss: 0.8603 - accuracy: 0.7804 - val_loss: 0.7353 - val_accuracy: 0.8034
Epoch 5/10
108/108 [==============================] - 3s 25ms/step - loss: 0.6551 - accuracy: 0.8180 - val_loss: 0.5847 - val_accuracy: 0.8320
Epoch 6/10
108/108 [==============================] - 3s 23ms/step - loss: 0.5373 - accuracy: 0.8425 - val_loss: 0.4975 - val_accuracy: 0.8516
Epoch 7/10
108/108 [==============================] - 2s 23ms/step - loss: 0.4616 - accuracy: 0.8613 - val_loss: 0.4283 - val_accuracy: 0.8712

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [32]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    #print("Debug input_shape =" , input_shape)
    #print("Debug output_sequence_length =" , output_sequence_length)
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)

    # Hyperparameters
    dropout = 0.0
    learning_rate = 1e-3

    # Choose Sequential or Functional API implementation ('seq' or 'func')
    impl='seq'
    if impl=='func':
        # Sequential Model
        print("Using Sequential model (Note: this version makes the unitary test to fail: Disable tests to use it)")
        from keras.models import Sequential
        model = Sequential()
        model.add(Bidirectional(GRU(english_vocab_size, dropout=dropout, return_sequences=True)))
        model.add(Dense(french_vocab_size, activation='softmax'))

    else:
        # model's Functional equivalent
        # Note : we could have also used "Bidirectional(GRU(...))" instead of buidling the Bidirectional RNNS manually
        print("Using Functional API")
        from keras.layers import concatenate, add
        input_seq = Input(shape=input_shape[1:])
        right_rnn = GRU(units=english_vocab_size, return_sequences=True, go_backwards=False)(input_seq)
        left_rnn = GRU(units=english_vocab_size, return_sequences=True, go_backwards=True)(input_seq)

        # Choose how to merge the 2 rnn layers : add or concatenate
        #logits = TimeDistributed(Dense(units=french_vocab_size))(add([right_rnn, left_rnn]))
        logits = TimeDistributed(Dense(units=french_vocab_size))(concatenate([right_rnn, left_rnn]))

        model = Model(input_seq, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])

    return model

#tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

# Pad and Reshape the input to work with a RNN without an Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
#print("Debug tmp_x shape=", tmp_x.shape )

# Train the neural network
bd_rnn_model = bd_model(input_shape = tmp_x.shape,
                           output_sequence_length = max_french_sequence_length,
                           english_vocab_size = english_vocab_size+1,
                           french_vocab_size = french_vocab_size+1)

#print(model.summary())

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Using Functional API


Epoch 1/10
108/108 [==============================] - 8s 43ms/step - loss: 2.5553 - accuracy: 0.4716 - val_loss: 1.9290 - val_accuracy: 0.5499
Epoch 2/10
108/108 [==============================] - 4s 34ms/step - loss: 1.6809 - accuracy: 0.5802 - val_loss: 1.5202 - val_accuracy: 0.5971
Epoch 3/10
108/108 [==============================] - 4s 36ms/step - loss: 1.4412 - accuracy: 0.6077 - val_loss: 1.3704 - val_accuracy: 0.6214
Epoch 4/10
108/108 [==============================] - 4s 36ms/step - loss: 1.3191 - accuracy: 0.6308 - val_loss: 1.2665 - val_accuracy: 0.6429
Epoch 5/10
108/108 [==============================] - 4s 39ms/step - loss: 1.2260 - accuracy: 0.6488 - val_loss: 1.1843 - val_accuracy: 0.6559
Epoch 6/10
108/108 [==============================] - 4s 36ms/step - loss: 1.1562 - accuracy: 0.6603 - val_loss: 1.1284 - val_accuracy: 0.6660
Epoch 7/10
108/108 [==============================] - 4s 33ms/step - loss: 1.1101 - accuracy: 0.6675 - val_loss: 1.0889 - val_accuracy: 0.6718

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [33]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement

    #print("Debug input_shape =" , input_shape, "  input_shape[1:] =", input_shape[1:], "  Input length=", input_shape[1:][0])
    #print("Debug output_sequence_length =" , output_sequence_length)
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)

    # Hyperparameters
    embedding_size = 128
    rnn_cells = 200
    dropout = 0.0
    learning_rate = 1e-3

    from keras.layers import LSTM

    # Input
    encoder_input_seq = Input(shape=input_shape[1:], name="enc_input")

    # Encoder (Return the internal states of the RNN -> 1 hidden state for GRU cells, 2 hidden states for LSTM cells))
    encoder_output, state_t = GRU(units=rnn_cells,
                                  dropout=dropout,
                                  return_sequences=False,
                                  return_state=True,
                                  name="enc_rnn")(encoder_input_seq)
          #or for LSTM cells: encoder_output, state_h, state_c = LSTM(...)

    # Decoder Input
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)

    # Decoder RNN (Take the encoder returned states as initial states)
    decoder_out = GRU(units=rnn_cells,
                      dropout=dropout,
                      return_sequences=True,
                      return_state=False)(decoder_input_seq, initial_state=state_t)
                                         #or for LSTM cells: (decoder_input_seq, initial_state=[state_h, state_c])

    # Decoder output
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_out)

    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])

    return model

# Unitary tests
#tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

# Pad and Reshape the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
#print("Debug tmp_x shape=", tmp_x.shape )

# Train the neural network
encdec_rnn_model = encdec_model(input_shape = tmp_x.shape,
                                output_sequence_length = max_french_sequence_length,
                                english_vocab_size = english_vocab_size+1,
                                french_vocab_size = french_vocab_size+1)

#print(encdec_rnn_model.summary())

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2) # callbacks=[tensorboard]

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))



Epoch 1/10
108/108 [==============================] - 8s 42ms/step - loss: 2.8478 - accuracy: 0.4569 - val_loss: 2.2320 - val_accuracy: 0.5018
Epoch 2/10
108/108 [==============================] - 4s 37ms/step - loss: 2.0345 - accuracy: 0.5328 - val_loss: 1.7863 - val_accuracy: 0.5594
Epoch 3/10
108/108 [==============================] - 4s 33ms/step - loss: 1.6707 - accuracy: 0.5712 - val_loss: 1.5903 - val_accuracy: 0.5797
Epoch 4/10
108/108 [==============================] - 4s 33ms/step - loss: 1.5471 - accuracy: 0.5855 - val_loss: 1.5109 - val_accuracy: 0.5891
Epoch 5/10
108/108 [==============================] - 4s 37ms/step - loss: 1.4745 - accuracy: 0.5981 - val_loss: 1.4353 - val_accuracy: 0.6083
Epoch 6/10
108/108 [==============================] - 4s 36ms/step - loss: 1.3994 - accuracy: 0.6136 - val_loss: 1.3570 - val_accuracy: 0.6246
Epoch 7/10
108/108 [==============================] - 4s 36ms/step - loss: 1.3283 - accuracy: 0.6278 - val_loss: 1.2941 - val_accuracy: 0.6335

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [34]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    # TODO: Train the final model
    #print("Debug input_shape =" , input_shape, "  input_shape[1:] =", input_shape[1:], "  Input length=", input_shape[1:][0])
    #print("Debug output_sequence_length =" , output_sequence_length)
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)


    ## Model_final is a seq2seq model (encoder-decoder) using embedding and bidirectional LSTM

    # Hyperparameters
    embedding_size = 128
    rnn_cells = 300
    dropout = 0.2
    learning_rate = 1e-3

    from keras.layers import LSTM, concatenate

    # Input and embedding
    encoder_input_seq = Input(shape=input_shape[1:])
    embedded_input_seq = Embedding(input_dim = english_vocab_size,
                                   output_dim = embedding_size,
                                   input_length=input_shape[1:][0])(encoder_input_seq)

    ## Note Bidirectional LSTM is used on the encoder side (see https://arxiv.org/pdf/1609.08144.pdf )
    ## Alternate version : Encoder RNN Bidirectional layer (Using the Keras Bidirectional layer wrappers)
    #encoder_output, forward_state_h, forward_state_c, backward_state_h, backward_state_c = Bidirectional(LSTM(units=rnn_cells,
    #                                                                                                          dropout=dropout,
    #                                                                                                          return_sequences=False,
    #                                                                                                          return_state=True))(embedded_input_seq)

    # Encoder Forward RNN layer
    encoder_forward_output, forward_state_h, forward_state_c = LSTM(units=rnn_cells,
                                                                    dropout=dropout,
                                                                    return_sequences=False,
                                                                    return_state=True,
                                                                    go_backwards=False)(embedded_input_seq)

    # Encoder backward RNN layer
    encoder_backward_output, backward_state_h, backward_state_c = LSTM(units=rnn_cells,
                                                                       dropout=dropout,
                                                                       return_sequences=False,
                                                                       return_state=True,
                                                                       go_backwards=True)(embedded_input_seq)

    # Encoder output and states : Merge the LSTM Forward and Backward ouputs (using 'concatenate' method)
    state_h = concatenate([forward_state_h, backward_state_h])
    state_c = concatenate([forward_state_c, backward_state_c])
    encoder_output = concatenate([encoder_forward_output, encoder_backward_output])

    # Decoder Input
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)

    # Decoder RNN layer
    # Note : we need twice more LSTM cells as we have concatenated backward and forward LSTM layers in encoder
    decoder_output = LSTM(units=rnn_cells*2,
                                  dropout=dropout,
                                  return_sequences=True,
                                  return_state=False,
                                  go_backwards=False)(decoder_input_seq, initial_state=[state_h, state_c])

    # Decoder output
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_output)

    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])

    return model

#tests.test_model_final(model_final)
print('Final Model Loaded\n')

# Train and Print prediction(s)

# Pad the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#print("Debug tmp_x shape=", tmp_x.shape )


# Train the neural network
final_rnn_model = model_final(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)

print(final_rnn_model.summary(line_length=125))


final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Final Model Loaded



Model: "model_5"
_____________________________________________________________________________________________________________________________
 Layer (type)                        Output Shape                          Param #     Connected to                          
 input_5 (InputLayer)                [(None, 21)]                          0           []                                    
                                                                                                                             
 embedding_2 (Embedding)             (None, 21, 128)                       25600       ['input_5[0][0]']                     
                                                                                                                             
 lstm (LSTM)                         [(None, 300),                         514800      ['embedding_2[0][0]']                 
                                      (None, 300),                                                   

1/1 [==============================] - 1s 913ms/step
new jersey est parfois calme pendant l' et il automne neigeux neigeux avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Prediction (IMPLEMENTATION)

In [35]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(input_shape = x.shape,
                        output_sequence_length = y.shape[1],
                        english_vocab_size = len(x_tk.word_index)+1,
                        french_vocab_size = len(y_tk.word_index)+1)

    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')

    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/20
108/108 [==============================] - 28s 213ms/step - loss: 2.5309 - accuracy: 0.4842 - val_loss: 1.6495 - val_accuracy: 0.5736
Epoch 2/20
108/108 [==============================] - 22s 206ms/step - loss: 1.4263 - accuracy: 0.6174 - val_loss: 1.2994 - val_accuracy: 0.6473
Epoch 3/20
108/108 [==============================] - 22s 207ms/step - loss: 1.1420 - accuracy: 0.6818 - val_loss: 1.0126 - val_accuracy: 0.7089
Epoch 4/20
108/108 [==============================] - 22s 201ms/step - loss: 0.9783 - accuracy: 0.7150 - val_loss: 0.9539 - val_accuracy: 0.7172
Epoch 5/20
108/108 [==============================] - 21s 195ms/step - loss: 0.8710 - accuracy: 0.7379 - val_loss: 0.7867 - val_accuracy: 0.7620
Epoch 6/20
108/108 [==============================] - 22s 206ms/step - loss: 0.7773 - accuracy: 0.7618 - val_loss: 0.6988 - val_accuracy: 0.7861
Epoch 7/20
108/108 [==============================] - 22s 204ms/step - loss: 0.6753 - accuracy: 0.7918 - val_loss: 0.6089 - val_ac

1/1 [==============================] - 1s 927ms/step
Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

In [55]:
from sklearn.model_selection import train_test_split

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    # TODO: Train neural network using model_final
    model = model_final(input_shape=x_train.shape,
                        output_sequence_length=y_train.shape[1],
                        english_vocab_size=len(x_tk.word_index) + 1,
                        french_vocab_size=len(y_tk.word_index) + 1)

    model.fit(x_train, y_train, batch_size=1024, epochs=20, validation_split=0.2)

    # Evaluate the model on the test data
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

     ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')

    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
    return model

model=final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/20
87/87 [==============================] - 27s 237ms/step - loss: 2.8335 - accuracy: 0.4526 - val_loss: 1.9475 - val_accuracy: 0.5352
Epoch 2/20
87/87 [==============================] - 19s 216ms/step - loss: 1.6502 - accuracy: 0.5752 - val_loss: 1.3858 - val_accuracy: 0.6265
Epoch 3/20
87/87 [==============================] - 18s 209ms/step - loss: 1.3028 - accuracy: 0.6408 - val_loss: 1.2192 - val_accuracy: 0.6582
Epoch 4/20
87/87 [==============================] - 17s 193ms/step - loss: 1.1625 - accuracy: 0.6724 - val_loss: 1.0623 - val_accuracy: 0.6965
Epoch 5/20
87/87 [==============================] - 18s 210ms/step - loss: 1.0356 - accuracy: 0.6992 - val_loss: 1.0022 - val_accuracy: 0.7056
Epoch 6/20
87/87 [==============================] - 17s 197ms/step - loss: 0.9460 - accuracy: 0.7196 - val_loss: 0.8881 - val_accuracy: 0.7339
Epoch 7/20
87/87 [==============================] - 18s 211ms/step - loss: 0.8586 - accuracy: 0.7405 - val_loss: 0.7954 - val_accuracy: 0.7569

In [69]:
import nltk
from nltk.corpus import stopwords

# Download the stopwords dataset (only need to do this once)
nltk.download('stopwords')

# Get the list of stopwords for a specific language, for example, English
stop_words = set(stopwords.words('english'))
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
from sklearn.model_selection import train_test_split

def final_predictions(x, y, x_tk, y_tk,model):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Split the data into training and testing sets
     ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = english_sentences[1000]
    sentence = [x_tk.word_index[word] for word in sentence.split() if word not in string.punctuation]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(english_sentences[1000])
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print(french_sentences[1000])


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer,model)

1/1 [==============================] - 0s 31ms/step
Sample 1:
they like strawberries , pears , and bananas .
ils aiment les fraises les poires et les bananes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
ils aiment les fraises , les poires et les bananes .
